<a href="https://colab.research.google.com/github/julioquevdo/LorchatbotIA/blob/main/chatbotIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pré-Requisitos para começar

In [ ]:
pip install -q -U google-generativeai pandas nltk gradio

In [ ]:
import nltk
nltk.download('punkt')

### ChatBot Educacional: Lor
Apenas substitua o *GOOGLE_API_KEY* para a sua API para começar

In [33]:
import google.generativeai as genai
import pandas as pd
import nltk
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

GOOGLE_API_KEY="[INSERIR AQUI]"
genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

# Criar o modelo AQUI, fora da função
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings,)

# Caminho do arquivo .txt
nome_arquivo = "/content/data/base_de_conhecimento.txt"

# Função para carregar a base de conhecimento
def carregar_base_conhecimento(nome_arquivo):
    with open(nome_arquivo, "r", encoding="utf-8") as f:
        texto = f.read()
    return texto

# Função para buscar conhecimento
def buscar_conhecimento(consulta, base_conhecimento):
    frases = nltk.sent_tokenize(base_conhecimento)
    vectorizer = TfidfVectorizer()
    vetores = vectorizer.fit_transform(frases)
    consulta_vetor = vectorizer.transform([consulta])
    similaridades = cosine_similarity(consulta_vetor, vetores).flatten()
    indice_mais_semelhante = similaridades.argmax()
    return frases[indice_mais_semelhante]

# Interface com Gradio
def interface_chatbot(entrada_usuario, nivel_complexidade):
    resposta = gerenciar_dialogo(entrada_usuario, base_conhecimento, nivel_complexidade)
    return resposta

iface = gr.Interface(
    fn=interface_chatbot,
    inputs=[
        gr.Textbox(label="Entrada"),
        gr.Radio(["iniciante", "intermediário", "avançado"], label="Nível de Complexidade")
    ],
    outputs=gr.Markdown(),
    title="Lor: Chatbot Educacional de IA"
)

# Função para gerenciar o diálogo
def gerenciar_dialogo(entrada_usuario, base_conhecimento, nivel_complexidade):

    # Instruções do sistema
    system_instruction = """[Gerar a resposta a partir do {estilo} que será fornecido posteriormente] # Configurações \n\nnome = \seu nome "Lor\"\npersonalidade:\nAmigável, paciente, informativa e entusiasmada com IA. \nGosta de ensinar e tornar o conhecimento sobre IA acessível a todos.\n\nconhecimento:\nVocê possui amplo conhecimento em inteligência artificial, machine learning e deep learning, incluindo:\n* Conceitos fundamentais \n* Técnicas e algoritmos\n* Aplicações em diferentes áreas\n* Tendências atuais e pesquisas recentes\n\n# Instruções\n\nVocê é a IA Educadora, uma especialista em inteligência artificial, machine learning e deep learning. Sua missão é:\n\n* Ensinar as pessoas sobre esses temas de forma clara, concisa e acessível.\n* Responder a perguntas, mesmo que sejam abertas, desafiadoras ou estranhas. \n* Explicar conceitos complexos de maneira simples e compreensível.\n* Fornecer exemplos relevantes e estudos de caso.\n* Discutir aplicações e implicações éticas da IA.\n* Aprender novas informações a partir de arquivos de texto e código.\n* Gerar diferentes respostas criativas de texto, como poemas, código, scripts, peças musicais, e-mail, cartas, etc.\n* Tentar seguir suas instruções e completar seus pedidos cuidadosamente.\n\n# Integração com o banco de dados\n\nVocê tem acesso a um banco de dados para auxiliar em suas tarefas. Utilize ele para:\n\n* Expandir seu conhecimento e se manter atualizada sobre as últimas pesquisas e desenvolvimentos em IA.\n* Gerar respostas mais completas e informativas.\n* Adaptar seu estilo de comunicação ao contexto da conversa.\n\n# Exemplo de Interação\n\n**Usuário:** O que é inteligência artificial?\n\n**IA Educadora:** A inteligência artificial (IA) é um campo amplo da ciência da computação que se concentra na criação de máquinas inteligentes que podem realizar tarefas que normalmente requerem inteligência humana, como aprendizado, resolução de problemas e tomada de decisões. \n\n**Usuário:** Você pode me dar alguns exemplos de como a IA está sendo usada hoje?\n\n**IA Educadora:** Claro! A IA está sendo usada em uma variedade de indústrias e aplicações, incluindo:\n\n* **Assistentes virtuais:** Como Siri e Alexa, que podem entender comandos de voz e realizar tarefas.\n* **Carros autônomos:** Que usam IA para navegar pelas estradas e evitar obstáculos.\n* **Diagnóstico médico:** Onde a IA pode ser usada para analisar imagens médicas e identificar doenças.\n* **Recomendações personalizadas:** Como as que você vê em plataformas de streaming como Netflix e Spotify.\n\n**Usuário:** Isso é fascinante! Eu quero aprender mais sobre IA.\n\n**IA Educadora:** Ótimo! Eu ficaria feliz em te ensinar mais sobre IA. O que você gostaria de aprender especificamente?\n\n# Lembre-se:\n\n* Seja sempre respeitosa e evite expressar opiniões pessoais ou crenças.\n* Mantenha o foco no tema da IA e evite divagações.\n* Se você não souber a resposta para uma pergunta, admita e ofereça-se para pesquisar mais informações."""

    # Buscar conhecimento relevante
    informacao = buscar_conhecimento(entrada_usuario, base_conhecimento)

    # Ajustar o estilo da resposta com base no nível de complexidade
    if nivel_complexidade == "iniciante":
        estilo = "Explique de forma simples usando analogias e sem muitos conceitos, explique de forma direta e sem muita complexidade. Evite usar muitos tópicos, perceba que deve ser fácil de ser entendido levando em consideração que não existem conhecimento prévio de quem fez a pergunta."""
    elif nivel_complexidade == "intermediário":
        estilo = "Forneça detalhes adicionais e exemplos, levando em consideração que quem fez a pergunta tem um pouco de conhecimento prévio e pode ser que saiba alguns conceitos básicos, caso você use algum conceito mais díficil, adicione um dicionário ao final da resposta."
    elif nivel_complexidade == "avançado":
        estilo = "Use termos técnicos e referências a pesquisas recentes, visto que quem fez essa pergunta é alguem com expertise na área. De respostas mais complexas e elaboradas, puxando conceitos que são vistos dentro da resposta do que foiperguntado"
    else:
        estilo = """Forneça detalhes adicionais e exemplos, levando em consideração que quem fez a pergunta tem um pouco de conhecimento prévio e pode ser que saiba alguns conceitos básicos, caso você use algum conceito mais díficil, adicione um dicionário ao final da resposta."""

    # Prompt
    prompt = f"""

    ## Sua Resposta (como IA Educadora)
    {estilo}

    {system_instruction}

    ## Contexto da Conversa
    Aqui está alguma informação relevante:
    {informacao}

    ## Pergunta do Usuário
    {entrada_usuario}
    """

    # Gerar resposta com o Gemini
    resposta_gemini = model.generate_content(prompt)
    return resposta_gemini.text

# Carregar a base de conhecimento
base_conhecimento = carregar_base_conhecimento(nome_arquivo)

iface.launch

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7e3816b8bdae4e8829.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://7e3816b8bdae4e8829.gradio.live


###Criador de Roteiro de Estudos
Apenas substitua o *GOOGLE_API_KEY* para a sua API para começar

In [34]:
import gradio as gr
import re
import google.generativeai as genai

# Substitua pela sua API Key
GOOGLE_API_KEY = "[INSERIR AQUI]"
genai.configure(api_key=GOOGLE_API_KEY)

# Configurações de geração
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

# Configurações de segurança
safety_settings = {
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE'
}

# Criar o modelo Gemini
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

import gradio as gr
import re

nome_arquivo= "/content/data/base_roteiro.txt"

# Carrega o arquivo base_roteiro.txt
def carregar_base(nome_arquivo):
    with open(nome_arquivo, 'r', encoding='utf-8') as f:
        conteudo = f.read()
    return conteudo

def extrair_topicos(conteudo):
    topicos = re.findall(r'\{(.*?)\}', conteudo, re.DOTALL)
    return topicos

def extrair_links(conteudo):
    links = re.findall(r'(https?://\S+)', conteudo)
    return links

# Cria o roteiro de estudos
def gerar_roteiro(topico_escolhido, topicos, links):
    roteiro = f"## Roteiro de Estudos: {topico_escolhido}\n\n"
    for i, topico in enumerate(topicos):
        if topico_escolhido.lower() in topico.lower():
            subtopicos = topico.split("\n")
            for subtopico in subtopicos:
                if subtopico.strip():
                    roteiro += f"**{subtopico.strip()}**\n"
            if links:
                roteiro += f"\n**Links Úteis:**\n"
                for link in links:
                    roteiro += f"- {link}\n"
            return roteiro
    return f"Desculpe, não encontrei informações sobre '{topico_escolhido}'."

# Interface com o Gradio
def interface_chatbot(topico):
    conteudo = carregar_base()
    topicos = extrair_topicos(conteudo)
    links = extrair_links(conteudo)
    roteiro = gerar_roteiro(topico, topicos, links)
    return roteiro


# Interface com o Gradio (modificada)
def interface_chatbot(topico):
  conteudo = carregar_base(nome_arquivo)
  topicos = extrair_topicos(conteudo)
  links = extrair_links(conteudo)
  roteiro = gerar_roteiro(topico, topicos, links)

  #Prompr
  prompt = f"""

  Crie um roteiro de estudos completo e informativo, em formato markdown, dizendo o porque aquilo deve ser aprendido e quais os pré requisitos para passar para uma nova fase, para o tópico '{topico}' (como o exemplo a seguir),
  incluindo os subtópicos e links fornecidos:

  {roteiro}

  ##exemplo de roteiro de estudos

  #substitua o campo (link) pelo link apropriado para o tópico desejado

  ## Exemplo de Roteiro de Estudos

**Fase 1: Fundamentos
*Fundamento 1
  *conceito do fundamento 1 (link)
  *conceitos do fundamento 1
  *conceitos do fundamento 1
  *conceitos do fundamento 1(link)

**Pré-requisitos para Fase 2:

**Fase 2: Análise de Dados e Web Scraping**

*Fundamento 1
  *conceito do fundamento 1(link)
  *conceitos do fundamento 1(link)
  *conceitos do fundamento 1(link)
  *conceitos do fundamento 1
*Fundamento 2
  *conceito do fundamento 2
  *conceitos do fundamento 2(link)
  *conceitos do fundamento 2(link)
  *conceitos do fundamento 2


**Dicas para se manter informado e continuar aprendendo:**

* **Siga especialistas e comunidades no Twitter:**
    *exemplo de twitter
* **Assine newsletters relevantes:**
  *exemplo de newsletters (link)
  *exemplo de newsletters
* **Leia artigos científicos e blogs:**
  *exemplo de artigos
  *exemplo de blogs
* **Participe de eventos e workshops:**
* **Contribua para projetos open-source:**

##Observações
Caso você não ter um link que seja pertinente, não precisa adicionar.
Siga essa estrutura, fases, pré-requisitos e dicas para se manter informado e continuar aprendendo. Diga também o porque aquilo deve ser aprendido.
Se o tópico que for requisitado não for relacionado a Inteligência Artificial, diga "Desculpe, não encontrei informações sobre '{topico}'."
  """

  resposta_gemini = model.generate_content(prompt)
  return resposta_gemini.text

iface = gr.Interface(
    fn=interface_chatbot,
    inputs=gr.Textbox(label="Digite o tópico desejado:"),
    outputs=gr.Markdown(),
    title="Chatbot de Roteiros de Estudos"
)

iface.launch

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e2b04a367164fa32ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://e2b04a367164fa32ec.gradio.live
